In [1]:
import numpy as np

In [10]:
import numpy as np

# Posture Feature: stride length

def compute_stride_length(sequence):
    """
    Input: sequence (30, 48) — 30 frames, 16 joints, each joint has 3D coords
    Output: stride length (float)
    """
    num_frames = sequence.shape[0]
    num_joints = 16
    coords = sequence.reshape(num_frames, num_joints, 3)

    idx_l_foot = 12  # index of left foot
    idx_r_foot = 15  # index of right foot

    dists = np.linalg.norm(coords[:, idx_l_foot, :] - coords[:, idx_r_foot, :], axis=1)
    stride_length = np.max(dists)
    return stride_length


# Per-frame features

def extract_features_per_frame(sequence):
    """
    sequence: np.ndarray with shape (30, 48)
    Each row is a frame, containing 16 joints with 3D coordinates in X,Z,Y order.
    Output: np.ndarray with shape (30, 13)
    """
    num_frames = sequence.shape[0]
    num_joints = 16
    joint_coords = sequence.reshape(num_frames, num_joints, 3)

    # Joint indices
    idx = {
        'root': 0,
        'spine': 1,
        'neck': 2,
        'head': 3,
        'l_shoulder': 4,
        'l_elbow': 5,
        'l_hand': 6,
        'r_shoulder': 7,
        'r_elbow': 8,
        'r_hand': 9,
        'l_hip': 10,
        'l_knee': 11,
        'l_foot': 12,
        'r_hip': 13,
        'r_knee': 14,
        'r_foot': 15
    }

    def angle_between(a, b, c):
        ba = a - b
        bc = c - b
        cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
        return np.degrees(angle)

    def triangle_area(a, b, c):
        ab = b - a
        ac = c - a
        return 0.5 * np.linalg.norm(np.cross(ab, ac))

    features = []

    for t in range(num_frames):
        joints = joint_coords[t]

        min_xyz = joints.min(axis=0)
        max_xyz = joints.max(axis=0)
        volume = np.prod(max_xyz - min_xyz)

        angle_neck_shoulders = angle_between(joints[idx['l_shoulder']], joints[idx['neck']], joints[idx['r_shoulder']])
        angle_r_shoulder = angle_between(joints[idx['neck']], joints[idx['r_shoulder']], joints[idx['l_shoulder']])
        angle_l_shoulder = angle_between(joints[idx['neck']], joints[idx['l_shoulder']], joints[idx['r_shoulder']])

        vertical = np.array([0, 1, 0])
        angle_neck_vertical_back = angle_between(joints[idx['neck']] + vertical, joints[idx['neck']], joints[idx['spine']])
        angle_neck_head_back = angle_between(joints[idx['head']], joints[idx['neck']], joints[idx['spine']])

        dist_r_hand_root = np.linalg.norm(joints[idx['r_hand']] - joints[idx['root']])
        dist_l_hand_root = np.linalg.norm(joints[idx['l_hand']] - joints[idx['root']])
        dist_r_foot_root = np.linalg.norm(joints[idx['r_foot']] - joints[idx['root']])
        dist_l_foot_root = np.linalg.norm(joints[idx['l_foot']] - joints[idx['root']])

        area_hands_neck = triangle_area(joints[idx['l_hand']], joints[idx['r_hand']], joints[idx['neck']])
        area_feet_root = triangle_area(joints[idx['l_foot']], joints[idx['r_foot']], joints[idx['root']])

        stride_length = np.linalg.norm(joints[idx['l_foot']] - joints[idx['r_foot']])

        features.append([
            volume,
            angle_neck_shoulders,
            angle_r_shoulder,
            angle_l_shoulder,
            angle_neck_vertical_back,
            angle_neck_head_back,
            dist_r_hand_root,
            dist_l_hand_root,
            dist_r_foot_root,
            dist_l_foot_root,
            stride_length,
            area_hands_neck,
            area_feet_root
        ])

    return np.array(features)  # (30, 13)


def extract_final_feature(sequence):
    """
    sequence: (30, 48)
    return: vector of length 14
    """
    stride = compute_stride_length(sequence)
    per_frame = extract_features_per_frame(sequence)
    avg_features = np.mean(per_frame, axis=0)  # shape (13,)
    return np.concatenate([avg_features, [stride]])  # shape (14,)


In [97]:
import h5py
import numpy as np

file_path = "data/Data.h5"

dataset_arrays = []  # List chứa từng dataset dạng numpy array (30, 48)

with h5py.File(file_path, "r") as f:
    def process_dataset(name, obj):
        if isinstance(obj, h5py.Dataset):
            data = obj[()]
            # Kiểm tra kích thước để đảm bảo (30,48)
            if data.shape == (30, 48):
                dataset_arrays.append(data)
            else:
                print(f"⚠ Dataset '{name}' có kích thước {data.shape}, bỏ qua.")

    f.visititems(process_dataset)

# Chuyển list thành numpy array (shape: [số dataset, 30, 48])
Gait = np.array(dataset_arrays)

print(f"Số dataset đã lấy: {Gait.shape[0]}")
print(f"Kích thước mảng cuối cùng: {Gait.shape}")


Số dataset đã lấy: 672
Kích thước mảng cuối cùng: (672, 30, 48)


In [50]:
Gait[0].shape[0]

30

In [76]:
Fm.shape

(672, 14)

In [31]:
def compute_movement_features(sequence):
    """
    Input:
        - sequence: np.ndarray of shape (30, 48) — 30 frames, 16 joints, mỗi joint có (X, Z, Y)
    Output:
        - movement features: np.ndarray of shape (16,)
    """

    num_frames = sequence.shape[0]
    num_joints = 16
    coords = sequence.reshape(num_frames, num_joints, 3)

    # Các chỉ số khớp
    idx_joints = {
        'r_hand': 9,
        'l_hand': 6,
        'head': 3,
        'r_foot': 15,
        'l_foot': 12
    }

    # (5, 30, 3)
    joint_trajs = np.stack([coords[:, idx, :] for idx in idx_joints.values()])

    # --- Tốc độ ---
    velocity = np.diff(joint_trajs, axis=1)                   # (5, 29, 3)
    speed = np.linalg.norm(velocity, axis=2)                  # (5, 29)
    avg_speed = np.mean(speed, axis=1)                        # (5,)

    # --- Gia tốc ---
    acceleration = np.diff(speed, axis=1)                     # (5, 28)
    avg_acceleration = np.mean(np.abs(acceleration), axis=1) # (5,)

    # --- Jerk ---
    jerk = np.diff(acceleration, axis=1)                      # (5, 27)
    avg_jerk = np.mean(np.abs(jerk), axis=1)                 # (5,)

    # --- Gait cycle time ---
    gait_cycle_time = num_frames / 30.0  # Giả sử 30 FPS

    # Gộp đặc trưng
    features = np.concatenate([
        avg_speed,
        avg_acceleration,
        avg_jerk,
        [gait_cycle_time]
    ])  # shape (16,)

    return features


In [77]:
Fm = []
Fp = []
for gait in Gait:
    temp_fm = extract_final_feature(gait)
    temp_fp = compute_movement_features(gait)
    Fm.append(temp_fm)
    Fp.append(temp_fp)
Fm = np.array(Fm)
Fp = np.array(Fp)


In [80]:
Fp.shape

(672, 16)

In [81]:
full_features = np.concatenate([Fm, Fp], axis = 1)

In [82]:
full_features.shape

(672, 30)

In [40]:
import torch
from lstm_model import VanillaLSTM
model = torch.load("lstm_model_full.pth", weights_only=False)
model = model.to("cpu")
lstm_layer = model.lstm
sequence = torch.tensor(Gait[0], dtype=torch.float32)  # sequence_np là (30, 48)
sequence = sequence.unsqueeze(1)  # Thêm batch dimension: (30, 1, 48)
output, (h_n, c_n) = lstm_layer(sequence)
final_hidden_state = h_n[-1]  # shape: (batch, hidden_size)
num_frames = Gait.shape[1]  # nếu Gait shape là (seq_len, features)
final_hidden = h_n.squeeze(0).squeeze(0)  # (hidden_size,) tensor có grad
lstm_out = final_hidden[num_frames-1].detach().cpu().numpy()  # convert sang numpy


In [89]:
model = torch.load("lstm_model_full.pth", weights_only=False)
model = model.to("cpu")
def deepfeatures(model, gait):
    lstm_layer = model.lstm
    sequence = torch.tensor(gait, dtype=torch.float32)  # sequence_np là (30, 48)
    sequence = sequence.unsqueeze(1)  # Thêm batch dimension: (30, 1, 48)
    output, (h_n, c_n) = lstm_layer(sequence)
    num_frames = gait.shape[0]  # nếu Gait shape là (seq_len, features)
    final_hidden = h_n.squeeze(0).squeeze(0)  # (hidden_size,) tensor có grad
    lstm_out = final_hidden[num_frames-1].detach().cpu().numpy()  # convert sang numpy
    return lstm_out

In [91]:
Fm = []
Fp = []
DeepFeatures = []
for gait in Gait:
    temp_fm = extract_final_feature(gait)
    temp_fp = compute_movement_features(gait)
    temp_deep = deepfeatures(model, gait)
    Fm.append(temp_fm)
    Fp.append(temp_fp)
    DeepFeatures.append(temp_deep)
Fm = np.array(Fm)
Fp = np.array(Fp)
DeepFeatures = np.array(DeepFeatures)


In [92]:
full_features = np.concatenate([Fm, Fp, DeepFeatures], axis = 1)


In [94]:
import h5py
import pandas as pd
import numpy as np

file_path = "data/Labels_My.h5"  # chỉnh đúng đường dẫn

row_1_values_all = []  # mảng chứa tất cả giá trị hàng 1 các dataset

with h5py.File(file_path, "r") as f:
    def process_dataset(name, obj):
        if isinstance(obj, h5py.Dataset):
            # Đọc toàn bộ dữ liệu dataset
            data = obj[()]
            df = pd.DataFrame(data)

            # Kiểm tra dataset có ít nhất 2 hàng không để lấy hàng 1
            if df.shape[0] > 1:
                # Lấy giá trị hàng 1 (index=1) dạng numpy array
                row_1 = df.iloc[1].values

                # Thêm từng phần tử trong hàng 1 vào mảng chung
                row_1_values_all.extend(row_1.tolist())
            else:
                print(f"⚠ Dataset {name} không có đủ 2 hàng để lấy hàng 1.")

    f.visititems(process_dataset)

# Chuyển list thành numpy array (tùy chọn)
labels = np.array(row_1_values_all)

print("Tổng giá trị hàng 1 của tất cả dataset gộp lại:")
print(labels.shape)

Tổng giá trị hàng 1 của tất cả dataset gộp lại:
(672,)


In [95]:
from sklearn.model_selection import train_test_split

X = full_features

# Ví dụ tạo label giả, vì bạn chưa cung cấp label (thường cần cho supervised learning)
# Đây chỉ là ví dụ: giả sử tất cả label là 0
y = labels

# Chia train-test (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True)


In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện
rf.fit(X_train, y_train)

# Dự đoán
y_pred = rf.predict(X_test)

# Đánh giá
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


🎯 Accuracy: 0.9259259259259259

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.95      0.92        19
           1       0.94      0.96      0.95        71
           2       0.85      0.81      0.83        27
           3       1.00      0.94      0.97        18

    accuracy                           0.93       135
   macro avg       0.92      0.92      0.92       135
weighted avg       0.93      0.93      0.93       135

